<a href="https://colab.research.google.com/github/AbelKristanto/machine_learning/blob/main/Build_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Remarks
In this notebook, we will learn more about "how to check fraud". We will learn tensorFlow. Let's started!

In [19]:
# import library data
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import functools
import pandas as pd
tf.executing_eagerly()

True

In [2]:
# prepare the data
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ls '/content/gdrive/My Drive/datasheet'

testing_data_fraud.csv	training_data_fraud.csv  validation_data_fraud.csv


In [10]:
train_data = pd.read_csv('/content/gdrive/My Drive/datasheet/training_data_fraud.csv')
test_data = pd.read_csv('/content/gdrive/My Drive/datasheet/testing_data_fraud.csv')
validation_data = pd.read_csv('/content/gdrive/My Drive/datasheet/validation_data_fraud.csv')

In [11]:
# define features and label
FEATURES = ['Time','V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24','V25','V26','V27','V28','Amount']
LABEL='Class'

In [12]:
# preprocess data 
MEANS = [94816.7387536405, 0.0011219465482001268, -0.0021445914636999603, -0.002317402958335562,
         -0.002525792169927835, -0.002136576923287782, -3.7586818983702984, 8.135919975738768E-4,
         -0.0015535579268265718, 0.001436137140461279, -0.0012193712736681508, -4.5364970422902533E-4,
         -4.6175444671576083E-4, 9.92177789685366E-4, 0.002366229151475428, 6.710217226762278E-4,
         0.0010325807119864225, 2.557260815835395E-4, -2.0804190062322664E-4, -5.057391100818653E-4,
         -3.452114767842334E-6, 1.0145936326270006E-4, 3.839214074518535E-4, 2.2061197469126577E-4,
         -1.5601580596677608E-4, -8.235017846415852E-4, -7.298316615408554E-4, -6.898459943652376E-5,
         4.724125688297753E-5, 88.73235686453587]
def norm_data(mean, data):
  data = tf.cast(data, tf.float32) * 1/(2*mean)
  return tf.reshape(data, [-1, 1])

numeric_columns = []

for i, feature in enumerate(FEATURES):
  num_col = tf.feature_column.numeric_column(feature, normalizer_fn=functools.partial(norm_data, MEANS[i]))
  numeric_columns.append(num_col)

numeric_columns

[NumericColumn(key='Time', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function norm_data at 0x7f4eb64a6c20>, 94816.7387536405)),
 NumericColumn(key='V1', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function norm_data at 0x7f4eb64a6c20>, 0.0011219465482001268)),
 NumericColumn(key='V2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function norm_data at 0x7f4eb64a6c20>, -0.0021445914636999603)),
 NumericColumn(key='V3', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function norm_data at 0x7f4eb64a6c20>, -0.002317402958335562)),
 NumericColumn(key='V4', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function norm_data at 0x7f4eb64a6c20>, -0.002525792169927835)),
 NumericColumn(key='V5', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function norm_data at 0x7f4eb64a6c

In [13]:
# create a model
model = keras.Sequential([
    tf.keras.layers.DenseFeatures(numeric_columns),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy', tf.keras.metrics.AUC(curve='PR')])

In [39]:
# train a model
CLASS_WEIGHT = {
    0: 1,
    1: 100
}
EPOCHS = 3

train_data = tf.convert_to_tensor(train_data)
val_data = tf.convert_to_tensor(validation_data)
test_data = tf.convert_to_tensor(test_data)

model.fit(train_data, validation_data=val_data, class_weight=CLASS_WEIGHT, epochs=EPOCHS)

ValueError: ignored

In [37]:
val_data

<tf.Tensor: shape=(28484, 31), dtype=float64, numpy=
array([[ 4.03000000e+02,  1.23741280e+00,  5.12364830e-01, ...,
         1.29863793e-02,  0.00000000e+00,  0.00000000e+00],
       [ 3.13600000e+03,  1.21619841e+00,  8.60135204e-02, ...,
        -1.78759690e-03,  0.00000000e+00,  0.00000000e+00],
       [ 3.34100000e+03, -2.86740520e-01,  1.17635466e+00, ...,
         1.29298053e-01,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 9.15540000e+04, -5.10025566e+00,  3.63344238e+00, ...,
         5.53580692e-01,  2.61220000e+02,  1.00000000e+00],
       [ 1.56685000e+05, -1.29777611e-01,  1.41547455e-01, ...,
         1.09191897e-01,  1.87110000e+02,  1.00000000e+00],
       [ 8.87370000e+04,  1.91782653e+00,  9.51966571e-01, ...,
        -1.21050907e-02,  9.21000000e+00,  1.00000000e+00]])>

In [36]:
train_data

<tf.Tensor: shape=(227896, 31), dtype=float64, numpy=
array([[ 2.82000000e+02, -3.56466190e-01,  7.25417515e-01, ...,
        -4.64013951e-02,  0.00000000e+00,  0.00000000e+00],
       [ 3.80000000e+02, -1.29983679e+00,  8.81817464e-01, ...,
         2.13787930e-02,  0.00000000e+00,  0.00000000e+00],
       [ 4.30000000e+02, -1.86025769e+00, -6.29858920e-01, ...,
        -2.23739480e-01,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 3.46340000e+04,  3.33499491e-01,  1.69987345e+00, ...,
         3.05473420e-01,  1.89600000e+01,  1.00000000e+00],
       [ 9.61350000e+04, -1.95293263e+00,  3.54138524e+00, ...,
         3.57891633e-01,  1.89600000e+01,  1.00000000e+00],
       [ 4.46200000e+03, -2.30334957e+00,  1.75924746e+00, ...,
        -1.53028797e-01,  2.39930000e+02,  1.00000000e+00]])>

In [26]:
# evaluate model
model.evaluate(test_data)

True